# Data Concatenation using Pandas - State Wise

In [1]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd 
from datetime import datetime, timedelta
import pickle


#### netcdf file reading

In [2]:
def unpack(path, lat_location, lon_location):
    # data = Dataset(path, 'r')
    try:
        data = Dataset(path, 'r')
    except Exception as e:
        print(f"An error occurred while opening the file: {str(e)}")
        return None  # Return None to indicate failure
    variable_name = list(data.variables.keys())[-1]
    # Storing the lat and lon data into the variables 
    lat = data.variables['lat'][:]
    lon = data.variables['lon'][:]


    # Squared difference of lat and lon 
    sq_diff_lat = (lat - lat_location)**2
    sq_diff_lon = (lon - lon_location)**2

    # Identifying the index of the minimum value for lat and lon 
    min_index_lat = sq_diff_lat.argmin()
    min_index_lon = sq_diff_lon.argmin()

    feature = data.variables[variable_name]

    days = data.variables['day']
    start_date = datetime(1900, 1, 1)  # Start date in the 1900 system
    dates = [start_date + timedelta(days=int(day)) for day in days]


    df = pd.DataFrame(columns=['Date', variable_name])
    df['Date'] = dates

    dt = np.arange(0, data.variables['day'].size)
    for time_index in dt:
        # Use numpy.ma.getdata to get unmasked values
        feature_values = feature[time_index, min_index_lat, min_index_lon]
        
        # Now, you can assign the unmasked values to the 'Temperature' column
        df.at[time_index, variable_name] = feature_values

    return df

#### file concatenation 

In [3]:
def create_feature_table(coordinates_df, file_paths: dict):
    final_dataframes = []

    for index, row in coordinates_df.iterrows():
        state = row['state'] # change to county 
        lat = row['Latitude']
        lon = row['Longitude']
        ID = row['ID']

        dfs = {}

        for data_type, paths in file_paths.items():
            data_type_dfs = []

            for path in paths:
                df = unpack(path, lat, lon)
                if df is None:
                    continue 
                variable_name = list(df.columns)[1]
                df.rename(columns={variable_name: data_type}, inplace=True)
                data_type_dfs.append(df)
                # print(path)
            # Concatenate the data for the current data_type
            data_type_df = pd.concat(data_type_dfs, axis=0, ignore_index=True)
            dfs[data_type] = data_type_df

        # Merge dataframes for different data_types
        final_df = dfs['min_humidity']  # Use one of the dataframes as a starting point
        for data_type, data_type_df in dfs.items():
            if data_type != 'min_humidity':
                final_df = pd.merge(final_df, data_type_df, on='Date', how='outer')

        final_df['state'] = state # change to county 
        final_df['ID'] = ID
        # print(final_df)
        final_dataframes.append(final_df)

    final_df = pd.concat(final_dataframes, axis=0, ignore_index=True)
    final_file_name = 'feature_df_all_counties.feather'
    # final_df.to_csv(final_file_name)
    # final_df.to_feather(final_file_name)
    return final_df


### Loading our nc file paths

In [4]:
with open('/Volumes/One Touch/BigDataProject-Capstone/dataCollection/file_path.pkl', 'rb') as pickle_file:
    loaded_dict = pickle.load(pickle_file)

In [5]:
coordinates_df = pd.read_csv("/Volumes/One Touch/BigDataProject-Capstone/dataCollection/state_coordinates.csv", index_col=False) # change to county 
coordinates_df

,ID,state,Latitude,Longitude
0,0,Illinois,40.6331,-89.3985
1,1,Indiana,40.2672,-86.1349
2,2,Iowa,41.8780,-93.0977
3,3,Kansas,38.5266,-96.7265
4,4,Minnesota,46.7296,-94.6859
5,5,Missouri,38.5739,-92.6032
6,6,Nebraska,41.4925,-99.9018
7,7,Ohio,40.4173,-82.9071
8,8,South Dakota,44.3683,-100.3509
9,9,Wisconsin,43.7844,-88.7879


In [6]:
coordinates = coordinates_df[:1]
coordinates

,ID,state,Latitude,Longitude
0,0,Illinois,40.6331,-89.3985


## Creating 1 consoilidated dataframe where 
#### - 9 columns represent the 9 weather variables
#### - the rows represent daily data from year 1980-2020
#### - we also have a ID/state column representing the daily weather variables for the respective state

In [6]:
df = create_feature_table(coordinates_df, loaded_dict)


In [7]:
df

,Date,min_humidity,max_humidity,min_temp,max_temp,vapor_pressure_deficit,near_surface_specific_humidity,precipitation,solar_radiation,wind_speed,state,ID
0,2000-01-01,55.1,100.0,267.0,280.6,0.16,0.00345,0.0,96.8,4.3,Illinois,0
1,2000-01-02,63.7,100.0,271.6,286.6,0.23,0.00511,0.0,94.1,5.9,Illinois,0
2,2000-01-03,59.0,100.0,268.9,277.7,0.05,0.00376,5.0,47.4,4.0,Illinois,0
3,2000-01-04,59.3,100.0,264.5,274.1,0.14,0.00219,0.0,78.3,7.3,Illinois,0
4,2000-01-05,43.4,88.9,263.8,273.7,0.2,0.0017,0.0,87.2,5.3,Illinois,0
...,...,...,...,...,...,...,...,...,...,...,...,...
194683,1999-12-27,31.4,86.0,266.5,275.8,0.24,0.00195,0.0,41.1,3.3,Kentucky,12
194684,1999-12-28,38.5,90.6,267.5,275.5,0.17,0.00245,0.0,67.8,5.4,Kentucky,12
194685,1999-12-29,54.4,96.5,268.5,280.3,0.13,0.00372,0.0,98.9,3.5,Kentucky,12
194686,1999-12-30,48.7,92.3,271.7,282.5,0.31,0.00346,0.0,83.9,3.1,Kentucky,12


In [8]:
import pickle
# Serialize and store the dictionary using pickle
with open('9_feature_1980_2020_statewise.pkl', 'wb') as pickle_file:
    pickle.dump(df, pickle_file)

In [ ]:
import pickle
# Serialize and store the dictionary using pickle
with open('final_df_1980_2020_statewise.pkl', 'wb') as pickle_file:
    pickle.dump(pivot_table, pickle_file)

# We group our daily date to - state and week wise. This week wise grouping is converted to feature columns

In [9]:
%%time
df = create_feature_table(coordinates_df, loaded_dict)
# df = data.copy()
df['Year'] = df['Date'].dt.year
# df['Week'] = df['Date'].dt.strftime('%U')
df['Week'] = df['Date'].dt.strftime('%W')
df_grouped = df.groupby(['Year', 'Week', 'state']).agg({'min_humidity': 'mean', 'max_humidity': 'mean','min_temp': 'mean', 'max_temp': 'mean','vapor_pressure_deficit': 'mean', 'near_surface_specific_humidity': 'mean','precipitation': 'mean', 'solar_radiation': 'mean','wind_speed':'mean'}).reset_index()
# df_grouped
pivot_table = df_grouped.pivot_table(index=['Year', 'state'], columns='Week', values=['min_humidity', 'max_humidity','min_temp','max_temp','vapor_pressure_deficit','near_surface_specific_humidity','precipitation','solar_radiation','wind_speed']).reset_index()
pivot_table.columns = [' '.join(col).strip() for col in pivot_table.columns.values]
pivot_table = pivot_table.rename(columns={'Year ': 'Year'})

CPU times: user 6min 18s, sys: 28.8 s, total: 6min 46s
Wall time: 20min 57s


In [18]:
pivot_table

,Year,state,max_humidity 00,max_humidity 01,max_humidity 02,max_humidity 03,max_humidity 04,max_humidity 05,max_humidity 06,max_humidity 07,...,wind_speed 44,wind_speed 45,wind_speed 46,wind_speed 47,wind_speed 48,wind_speed 49,wind_speed 50,wind_speed 51,wind_speed 52,wind_speed 53
0,1980,Illinois,94.300000,84.142857,99.057143,83.914286,79.671429,89.857143,91.385714,94.585714,...,5.442857,4.000000,4.600000,4.785714,4.242857,3.671429,3.885714,4.271429,4.366667,NaN
1,1980,Indiana,97.216667,85.142857,97.514286,90.371429,81.071429,97.414286,94.228571,98.357143,...,5.342857,3.714286,4.200000,4.871429,4.242857,3.828571,3.642857,3.657143,3.566667,NaN
2,1980,Iowa,85.600000,75.457143,83.857143,75.900000,73.400000,88.042857,86.042857,92.928571,...,4.600000,3.671429,4.585714,4.085714,4.071429,3.528571,3.857143,4.657143,4.033333,NaN
3,1980,Kansas,89.833333,75.200000,87.842857,83.800000,82.628571,88.871429,87.471429,89.000000,...,4.242857,5.428571,4.800000,4.128571,5.314286,3.857143,4.785714,5.128571,4.333333,NaN
4,1980,Kentucky,94.783333,86.214286,93.114286,86.871429,85.757143,88.100000,84.371429,89.928571,...,4.657143,3.742857,3.057143,5.114286,3.671429,3.657143,3.757143,3.514286,3.233333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,2020,Nebraska,84.580000,87.757143,92.057143,84.700000,82.428571,90.042857,90.942857,92.528571,...,6.171429,5.300000,3.885714,4.971429,4.228571,4.614286,4.257143,5.342857,4.150000,NaN
529,2020,North Dakota,81.540000,86.800000,96.100000,85.000000,67.942857,71.228571,87.557143,78.657143,...,5.814286,6.114286,4.314286,5.985714,4.714286,3.800000,5.328571,5.271429,5.250000,NaN
530,2020,Ohio,85.680000,82.428571,91.785714,86.985714,80.342857,87.585714,92.400000,76.900000,...,4.885714,5.400000,6.157143,4.257143,4.928571,4.542857,3.742857,6.728571,5.200000,NaN
531,2020,South Dakota,82.960000,79.428571,82.800000,74.914286,68.528571,79.142857,83.814286,75.800000,...,4.028571,3.842857,3.185714,3.900000,3.414286,3.185714,3.257143,3.842857,2.925000,NaN


In [11]:
import pickle
# Serialize and store the dictionary using pickle
with open('final_df_1980_2020_statewise.pkl', 'wb') as pickle_file:
    pickle.dump(pivot_table, pickle_file)

## Concatenation of Crop Yield column to get our target variable column in our data frame

In [12]:
corn_yield_data = pd.read_csv("/Volumes/One Touch/BigDataProject-Capstone/dataCollection/corn.csv")
corn_yield_data['State'] = corn_yield_data['State'].str.title()
corn_yield_data.rename(columns={'Value':'Corn_Yield'}, inplace=True)
corn_yield_data

,Year,State,Corn_Yield
0,2020,Illinois,191
1,2020,Indiana,187
2,2020,Iowa,177
3,2020,Kansas,134
4,2020,Kentucky,184
...,...,...,...
528,1980,Nebraska,85
529,1980,North Dakota,58
530,1980,Ohio,113
531,1980,South Dakota,53


In [13]:
corn_result = pivot_table.merge(corn_yield_data, left_on=['Year', 'state'], right_on=['Year', 'State'], how='left')
corn_result.drop(['State'],axis=1, inplace=True)
corn_result

,Year,state,max_humidity 00,max_humidity 01,max_humidity 02,max_humidity 03,max_humidity 04,max_humidity 05,max_humidity 06,max_humidity 07,...,wind_speed 45,wind_speed 46,wind_speed 47,wind_speed 48,wind_speed 49,wind_speed 50,wind_speed 51,wind_speed 52,wind_speed 53,Corn_Yield
0,1980,Illinois,94.300000,84.142857,99.057143,83.914286,79.671429,89.857143,91.385714,94.585714,...,4.000000,4.600000,4.785714,4.242857,3.671429,3.885714,4.271429,4.366667,NaN,93
1,1980,Indiana,97.216667,85.142857,97.514286,90.371429,81.071429,97.414286,94.228571,98.357143,...,3.714286,4.200000,4.871429,4.242857,3.828571,3.642857,3.657143,3.566667,NaN,96
2,1980,Iowa,85.600000,75.457143,83.857143,75.900000,73.400000,88.042857,86.042857,92.928571,...,3.671429,4.585714,4.085714,4.071429,3.528571,3.857143,4.657143,4.033333,NaN,110
3,1980,Kansas,89.833333,75.200000,87.842857,83.800000,82.628571,88.871429,87.471429,89.000000,...,5.428571,4.800000,4.128571,5.314286,3.857143,4.785714,5.128571,4.333333,NaN,94
4,1980,Kentucky,94.783333,86.214286,93.114286,86.871429,85.757143,88.100000,84.371429,89.928571,...,3.742857,3.057143,5.114286,3.671429,3.657143,3.757143,3.514286,3.233333,NaN,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,2020,Nebraska,84.580000,87.757143,92.057143,84.700000,82.428571,90.042857,90.942857,92.528571,...,5.300000,3.885714,4.971429,4.228571,4.614286,4.257143,5.342857,4.150000,NaN,180
529,2020,North Dakota,81.540000,86.800000,96.100000,85.000000,67.942857,71.228571,87.557143,78.657143,...,6.114286,4.314286,5.985714,4.714286,3.800000,5.328571,5.271429,5.250000,NaN,139
530,2020,Ohio,85.680000,82.428571,91.785714,86.985714,80.342857,87.585714,92.400000,76.900000,...,5.400000,6.157143,4.257143,4.928571,4.542857,3.742857,6.728571,5.200000,NaN,171
531,2020,South Dakota,82.960000,79.428571,82.800000,74.914286,68.528571,79.142857,83.814286,75.800000,...,3.842857,3.185714,3.900000,3.414286,3.185714,3.257143,3.842857,2.925000,NaN,162


In [14]:
import pickle
# Serialize and store the dictionary using pickle
with open('final_df_1980_2020_statewise_with_corn_yield.pkl', 'wb') as pickle_file:
    pickle.dump(corn_result, pickle_file)

In [15]:
soya_bean_yield_data = pd.read_csv("/Volumes/One Touch/BigDataProject-Capstone/dataCollection/soybean.csv")
soya_bean_yield_data['State'] = soya_bean_yield_data['State'].str.title()
soya_bean_yield_data.rename(columns={'Value':'SoyaBean_Yield'}, inplace=True)
soya_bean_yield_data

,Year,State,SoyaBean_Yield
0,2020,Illinois,60.0
1,2020,Indiana,59.0
2,2020,Iowa,54.0
3,2020,Kansas,41.0
4,2020,Kentucky,55.0
...,...,...,...
528,1980,Nebraska,30.0
529,1980,North Dakota,17.5
530,1980,Ohio,36.0
531,1980,South Dakota,26.0


In [16]:
soya_result = pivot_table.merge(soya_bean_yield_data, left_on=['Year', 'state'], right_on=['Year', 'State'], how='left')
soya_result.drop(['State'],axis=1, inplace=True)
soya_result

,Year,state,max_humidity 00,max_humidity 01,max_humidity 02,max_humidity 03,max_humidity 04,max_humidity 05,max_humidity 06,max_humidity 07,...,wind_speed 45,wind_speed 46,wind_speed 47,wind_speed 48,wind_speed 49,wind_speed 50,wind_speed 51,wind_speed 52,wind_speed 53,SoyaBean_Yield
0,1980,Illinois,94.300000,84.142857,99.057143,83.914286,79.671429,89.857143,91.385714,94.585714,...,4.000000,4.600000,4.785714,4.242857,3.671429,3.885714,4.271429,4.366667,NaN,33.5
1,1980,Indiana,97.216667,85.142857,97.514286,90.371429,81.071429,97.414286,94.228571,98.357143,...,3.714286,4.200000,4.871429,4.242857,3.828571,3.642857,3.657143,3.566667,NaN,36.0
2,1980,Iowa,85.600000,75.457143,83.857143,75.900000,73.400000,88.042857,86.042857,92.928571,...,3.671429,4.585714,4.085714,4.071429,3.528571,3.857143,4.657143,4.033333,NaN,38.5
3,1980,Kansas,89.833333,75.200000,87.842857,83.800000,82.628571,88.871429,87.471429,89.000000,...,5.428571,4.800000,4.128571,5.314286,3.857143,4.785714,5.128571,4.333333,NaN,16.5
4,1980,Kentucky,94.783333,86.214286,93.114286,86.871429,85.757143,88.100000,84.371429,89.928571,...,3.742857,3.057143,5.114286,3.671429,3.657143,3.757143,3.514286,3.233333,NaN,22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,2020,Nebraska,84.580000,87.757143,92.057143,84.700000,82.428571,90.042857,90.942857,92.528571,...,5.300000,3.885714,4.971429,4.228571,4.614286,4.257143,5.342857,4.150000,NaN,58.0
529,2020,North Dakota,81.540000,86.800000,96.100000,85.000000,67.942857,71.228571,87.557143,78.657143,...,6.114286,4.314286,5.985714,4.714286,3.800000,5.328571,5.271429,5.250000,NaN,34.0
530,2020,Ohio,85.680000,82.428571,91.785714,86.985714,80.342857,87.585714,92.400000,76.900000,...,5.400000,6.157143,4.257143,4.928571,4.542857,3.742857,6.728571,5.200000,NaN,55.0
531,2020,South Dakota,82.960000,79.428571,82.800000,74.914286,68.528571,79.142857,83.814286,75.800000,...,3.842857,3.185714,3.900000,3.414286,3.185714,3.257143,3.842857,2.925000,NaN,46.0


In [17]:
import pickle
# Serialize and store the dictionary using pickle
with open('final_df_1980_2020_statewise_with_soya_yield.pkl', 'wb') as pickle_file:
    pickle.dump(soya_result, pickle_file)